# Financial Base Table

#### Check for library requirements & install if missing

In [1]:
import sys
import subprocess
import importlib

packages = ['pandas', 'numpy','pathlib','os','glob', 'textblob']
[subprocess.check_call(['pip', 'install', pkg]) 
for pkg in packages if not importlib.util.find_spec(pkg)]

[]

#### Import libraries

In [2]:
import pandas as pd
import numpy as np

from pathlib import Path
import pathlib
import os
import glob

from textblob import TextBlob

#### Set working directory

In [3]:
data_folder = Path(os.getcwd() + "/FP_GroupProject/data/raw/data_berka")

In [4]:
# Change to working dir
os.chdir(data_folder)

#### Read data

In [5]:
# Loop through data folder and add all df's in a dict
data_dict={}
for txt_file in glob.glob("*.asc"):
    #list filenames
    filename = txt_file
    #create python tablenames
    df_name = str.replace(txt_file,".asc","_raw") 
    #read files
    df_value = pd.read_csv(filename,  delimiter=";")
    #add to dict
    data_dict[df_name] = df_value


/Users/inder/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# Split dict into indivisual dataframes
for var in data_dict.keys():
    exec("{} = data_dict['{}']".format(var, var))

In [7]:
data_dict.keys()

dict_keys(['account_raw', 'card_raw', 'order_raw', 'disp_raw', 'loan_raw', 'client_raw', 'district_raw', 'trans_raw'])

In [8]:
# Check for null values
for keys in data_dict.keys():
    print(keys, ":", data_dict[keys].isna().sum().sum())

account_raw : 0
card_raw : 0
order_raw : 0
disp_raw : 0
loan_raw : 0
client_raw : 0
district_raw : 0
trans_raw : 2208738


### Data exploration

In [9]:
account_raw.head()

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102


In [10]:
# Translate frequency column to english
freq_dict = {}
for text in account_raw.frequency.unique():
    freq_dict[text] =  (TextBlob(text).translate(to='en').raw)

In [11]:
freq_dict

{'POPLATEK MESICNE': 'MONTHLY FEE',
 'POPLATEK PO OBRATU': 'TURNOVER FEE',
 'POPLATEK TYDNE': 'FEE OF THE WEEK'}

In [12]:
#add translated values in dataframe
account_raw.frequency.replace({"POPLATEK MESICNE": freq_dict["POPLATEK MESICNE"],
                                "POPLATEK PO OBRATU": freq_dict["POPLATEK PO OBRATU"],
                                "POPLATEK TYDNE": freq_dict["POPLATEK TYDNE"]}, inplace=True)

In [25]:
account_raw.head(15)

,account_id,district_id,frequency,date
0,576,55,MONTHLY FEE,930101
1,3818,74,MONTHLY FEE,930101
2,704,55,MONTHLY FEE,930101
3,2378,16,MONTHLY FEE,930101
4,2632,24,MONTHLY FEE,930102
5,1972,77,MONTHLY FEE,930102
6,1539,1,TURNOVER FEE,930103
7,793,47,MONTHLY FEE,930103
8,2484,74,MONTHLY FEE,930103
9,1695,76,MONTHLY FEE,930103


In [26]:
#Convert date format
account_raw["date"] = pd.to_datetime("19" + account_raw["date"].astype(str), format='%Y%m%d')

In [36]:
#Split year, month and day
account_raw['acc_open_year'] = pd.DatetimeIndex(account_raw['date']).year
account_raw['acc_open_month'] = pd.DatetimeIndex(account_raw['date']).month
account_raw['acc_open_day'] = pd.DatetimeIndex(account_raw['date']).day

In [41]:
account = account_raw[account_raw['acc_open_year'] < 1996].copy(deep=True)

In [44]:
account.acc_open_year.unique()

array([1993, 1994, 1995])

In [45]:
#Add LOR
account['LOR'] = 1996 - account['acc_open_year']
account.head()

,account_id,district_id,frequency,date,acc_open_year,acc_open_month,acc_open_day,LOR
0,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3
1,3818,74,MONTHLY FEE,1993-01-01,1993,1,1,3
2,704,55,MONTHLY FEE,1993-01-01,1993,1,1,3
3,2378,16,MONTHLY FEE,1993-01-01,1993,1,1,3
4,2632,24,MONTHLY FEE,1993-01-02,1993,1,2,3


In [46]:
# Add information about account owner
account = pd.merge(account, disp_raw[disp_raw['type'] == 'OWNER'], how='left', on='account_id')
account = pd.merge(account, client_raw, how='left', on='client_id')
account = account.rename(columns={'district_id_x':'bank_district_id',
                        'district_id_y':'client_district_id'})
account.head()

,account_id,bank_district_id,frequency,date,acc_open_year,acc_open_month,acc_open_day,LOR,disp_id,client_id,type,birth_number,client_district_id
0,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,OWNER,365111,74
1,3818,74,MONTHLY FEE,1993-01-01,1993,1,1,3,4601,4601,OWNER,350402,1
2,704,55,MONTHLY FEE,1993-01-01,1993,1,1,3,844,844,OWNER,450114,22
3,2378,16,MONTHLY FEE,1993-01-01,1993,1,1,3,2873,2873,OWNER,755324,16
4,2632,24,MONTHLY FEE,1993-01-02,1993,1,2,3,3177,3177,OWNER,380812,24


In [50]:
# Transform the birth day into year
account['birth_year'] = '19' + account['birth_number'].astype(str).str[:2]
account['birth_year'] = account['birth_year'].astype(int)

# Transform the birth day to day
account['birth_day'] = account['birth_number'].astype(str).str[-2:].astype(int)

# Extract the birth month
account['birth_month'] = account['birth_number'].astype(str).str[2:4].astype(int)

In [53]:
# Extract and correct the gender
account['gender'] = 'M'
account.loc[account['birth_month'] > 50, 'gender'] = 'F'

# Correct the birth month
account.loc[account['birth_month'] > 50, 'birth_month'] = account.loc[account['birth_month'] > 50, 'birth_month'] - 50
account.head()

,account_id,bank_district_id,frequency,date,acc_open_year,acc_open_month,acc_open_day,LOR,disp_id,client_id,type,birth_number,client_district_id,birth_year,birth_day,birth_month,gender
0,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,OWNER,365111,74,1936,11,1,M
1,3818,74,MONTHLY FEE,1993-01-01,1993,1,1,3,4601,4601,OWNER,350402,1,1935,2,4,M
2,704,55,MONTHLY FEE,1993-01-01,1993,1,1,3,844,844,OWNER,450114,22,1945,14,1,M
3,2378,16,MONTHLY FEE,1993-01-01,1993,1,1,3,2873,2873,OWNER,755324,16,1975,24,3,M
4,2632,24,MONTHLY FEE,1993-01-02,1993,1,2,3,3177,3177,OWNER,380812,24,1938,12,8,M


In [54]:
# Age
account['age'] = 1996 - account['birth_year']

# Age group
account['age_group'] = account['age'] // 10 * 10
account.head()

,account_id,bank_district_id,frequency,date,acc_open_year,acc_open_month,acc_open_day,LOR,disp_id,client_id,type,birth_number,client_district_id,birth_year,birth_day,birth_month,gender,age,age_group
0,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,OWNER,365111,74,1936,11,1,M,60,60
1,3818,74,MONTHLY FEE,1993-01-01,1993,1,1,3,4601,4601,OWNER,350402,1,1935,2,4,M,61,60
2,704,55,MONTHLY FEE,1993-01-01,1993,1,1,3,844,844,OWNER,450114,22,1945,14,1,M,51,50
3,2378,16,MONTHLY FEE,1993-01-01,1993,1,1,3,2873,2873,OWNER,755324,16,1975,24,3,M,21,20
4,2632,24,MONTHLY FEE,1993-01-02,1993,1,2,3,3177,3177,OWNER,380812,24,1938,12,8,M,58,50


In [56]:
#Drop unwanted columns
account_dropped_cols = ["birth_number","date"]
account.drop(account_dropped_cols, axis=1)

,account_id,bank_district_id,frequency,date,acc_open_year,acc_open_month,acc_open_day,LOR,disp_id,client_id,type,birth_number,client_district_id,birth_year,birth_day,birth_month,gender,age,age_group
0,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,OWNER,365111,74,1936,11,1,M,60,60
1,3818,74,MONTHLY FEE,1993-01-01,1993,1,1,3,4601,4601,OWNER,350402,1,1935,2,4,M,61,60
2,704,55,MONTHLY FEE,1993-01-01,1993,1,1,3,844,844,OWNER,450114,22,1945,14,1,M,51,50
3,2378,16,MONTHLY FEE,1993-01-01,1993,1,1,3,2873,2873,OWNER,755324,16,1975,24,3,M,21,20
4,2632,24,MONTHLY FEE,1993-01-02,1993,1,2,3,3177,3177,OWNER,380812,24,1938,12,8,M,58,50


In [99]:
#Add age group description column
age_bkt = [
    (account["age"].between(0, 17)),
    (account["age"].between(18, 35)),
    (account["age"].between(36, 55)),
    (account["age"] > 56)
    ]

# create a list of the value for each condition
age_bkt_vals = ["youth", "young adult", "adult", "senior"]

In [103]:
account["age_grp_desc"] = np.select(age_bkt, age_bkt_vals)
account.head()

,account_id,bank_district_id,frequency,date,acc_open_year,acc_open_month,acc_open_day,LOR,disp_id,client_id,...,birth_number,client_district_id,birth_year,birth_day,birth_month,gender,age,age_group,cat,age_grp_desc
0,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,...,365111,74,1936,11,1,M,60,60,senior,senior
1,3818,74,MONTHLY FEE,1993-01-01,1993,1,1,3,4601,4601,...,350402,1,1935,2,4,M,61,60,senior,senior
2,704,55,MONTHLY FEE,1993-01-01,1993,1,1,3,844,844,...,450114,22,1945,14,1,M,51,50,adult,adult
3,2378,16,MONTHLY FEE,1993-01-01,1993,1,1,3,2873,2873,...,755324,16,1975,24,3,M,21,20,young adult,young adult
4,2632,24,MONTHLY FEE,1993-01-02,1993,1,2,3,3177,3177,...,380812,24,1938,12,8,M,58,50,senior,senior


### Transaction table

In [108]:
trans_raw

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1056315,3626622,2906,981231,PRIJEM,NaN,62.3,13729.4,UROK,NaN,NaN
1056316,3627616,2935,981231,PRIJEM,NaN,81.3,19544.9,UROK,NaN,NaN
1056317,3625403,2869,981231,PRIJEM,NaN,60.2,14638.2,UROK,NaN,NaN
1056318,3626683,2907,981231,PRIJEM,NaN,107.5,23453.0,UROK,NaN,NaN


In [107]:
trans_raw.isna().sum()

trans_id           0
account_id         0
date               0
type               0
operation     183114
amount             0
balance            0
k_symbol      481881
bank          782812
account       760931
dtype: int64

In [112]:
# create a dictionary to identify credit and withdrawl
type_dict = {'PRIJEM':'credit','VYDAJ':'withdrawl', 'VYBER':'withdrawl' }

In [114]:
#map dictionary to base dataframe for credit and withdrawl
trans_raw['trans_type_en'] = trans_raw['type'].map(type_dict)

In [115]:
trans_raw

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account,trans_type_en
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,credit
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,credit
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,credit
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,credit
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,credit
...,...,...,...,...,...,...,...,...,...,...,...
1056315,3626622,2906,981231,PRIJEM,NaN,62.3,13729.4,UROK,NaN,NaN,credit
1056316,3627616,2935,981231,PRIJEM,NaN,81.3,19544.9,UROK,NaN,NaN,credit
1056317,3625403,2869,981231,PRIJEM,NaN,60.2,14638.2,UROK,NaN,NaN,credit
1056318,3626683,2907,981231,PRIJEM,NaN,107.5,23453.0,UROK,NaN,NaN,credit


In [117]:
# Select on transactions for 1996
txn_96 = trans_raw[trans_raw['date'].astype(str).str[:2].isin(['96'])]

In [121]:
# Aggregate the total credit per account
trans_agg_credit = txn_96[txn_96['type'].isin(['PRIJEM'])].groupby('account_id')['amount'].agg('sum')
trans_agg_credit = trans_agg_credit.reset_index()
trans_agg_credit = trans_agg_credit.rename(columns={'amount':'total_credit'})


# Aggregate the total withdrawal per account
trans_agg_withdrawal = txn_96[txn_96['type'].isin(['VYDAJ', 'VYBER'])].groupby('account_id')['amount'].agg('sum')
trans_agg_withdrawal = trans_agg_withdrawal.reset_index()
trans_agg_withdrawal = trans_agg_withdrawal.rename(columns={'amount':'total_withdrawal'})


# Join with previous table
trans = pd.merge(trans_raw, trans_agg_credit, how='left', on='account_id')
trans = pd.merge(trans, trans_agg_withdrawal, how='left', on='account_id')
trans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account,trans_type_en,total_credit,total_withdrawal
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,credit,664545.4,633310.2
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,credit,76097.3,70419.2
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,credit,228514.9,218531.2
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,credit,234806.4,223535.2
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,credit,64589.5,60917.2


In [126]:
pd.merge(account[account["account_id"]==576], trans, how='left', on='account_id')

,account_id,bank_district_id,frequency,date_x,acc_open_year,acc_open_month,acc_open_day,LOR,disp_id,client_id,...,type_y,operation,amount,balance,k_symbol,bank,account,trans_type_en,total_credit,total_withdrawal
0,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,...,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,credit,76097.3,70419.2
1,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,...,PRIJEM,PREVOD Z UCTU,6207.0,7107.0,DUCHOD,YZ,30300313.0,credit,76097.3,70419.2
2,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,...,PRIJEM,NaN,20.1,7127.1,UROK,NaN,NaN,credit,76097.3,70419.2
3,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,...,PRIJEM,PREVOD Z UCTU,6207.0,13334.1,DUCHOD,YZ,30300313.0,credit,76097.3,70419.2
4,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,...,PRIJEM,NaN,29.6,13363.7,UROK,NaN,NaN,credit,76097.3,70419.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,...,VYDAJ,VYBER,300.0,34258.5,NaN,NaN,NaN,withdrawl,76097.3,70419.2
354,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,...,PRIJEM,PREVOD Z UCTU,6207.0,40465.5,DUCHOD,YZ,30300313.0,credit,76097.3,70419.2
355,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,...,VYDAJ,PREVOD NA UCET,3662.0,36803.5,SIPO,OP,71033382.0,withdrawl,76097.3,70419.2
356,576,55,MONTHLY FEE,1993-01-01,1993,1,1,3,692,692,...,VYDAJ,VYBER,1400.0,35403.5,NaN,NaN,NaN,withdrawl,76097.3,70419.2


In [122]:
#Drop unwanted columns
account_dropped_cols = ["birth_number","date"]
account.drop(account_dropped_cols, axis=1)

,account_id,bank_district_id,frequency,acc_open_year,acc_open_month,acc_open_day,LOR,disp_id,client_id,type,client_district_id,birth_year,birth_day,birth_month,gender,age,age_group,cat,age_grp_desc
0,576,55,MONTHLY FEE,1993,1,1,3,692,692,OWNER,74,1936,11,1,M,60,60,senior,senior
1,3818,74,MONTHLY FEE,1993,1,1,3,4601,4601,OWNER,1,1935,2,4,M,61,60,senior,senior
2,704,55,MONTHLY FEE,1993,1,1,3,844,844,OWNER,22,1945,14,1,M,51,50,adult,adult
3,2378,16,MONTHLY FEE,1993,1,1,3,2873,2873,OWNER,16,1975,24,3,M,21,20,young adult,young adult
4,2632,24,MONTHLY FEE,1993,1,2,3,3177,3177,OWNER,24,1938,12,8,M,58,50,senior,senior
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2234,4462,73,FEE OF THE WEEK,1995,12,27,1,5384,5384,OWNER,73,1935,21,7,M,61,60,senior,senior
2235,3814,74,MONTHLY FEE,1995,12,27,1,4596,4596,OWNER,74,1973,31,8,M,23,20,young adult,young adult
2236,2780,63,MONTHLY FEE,1995,12,29,1,3357,3357,OWNER,63,1954,21,7,M,42,40,adult,adult
2237,3273,74,MONTHLY FEE,1995,12,29,1,3962,3962,OWNER,74,1952,28,11,M,44,40,adult,adult
